```
Check for duplicates (Primary Key) .
```

In [ ]:
SELECT
    location_id, COUNT(1) AS numbers_of_records
FROM
    OPENROWSET(
        BULK 'abfss://nyc-taxi-data@wsmaindl.dfs.core.windows.net/raw/taxi_zone.csv',
        FORMAT = 'CSV',
        PARSER_VERSION = '2.0',
        FIRSTROW = 2,
        FIELDTERMINATOR = ',',
        ROWTERMINATOR = '\n'
    ) 
    WITH (
        location_id SMALLINT 1,
        borough VARCHAR(15) 2,
        zone VARCHAR(50) 3,
        service_zone  VARCHAR(15) 4 
    )AS [result]
    GROUP BY location_id
    HAVING COUNT(1) > 1;

Data quality checks for trip totoal amount

In [ ]:
SELECT
    MAX(total_amount) AS max_total_amount,
    MIN(total_amount) AS min_total_amount,
    AVG(total_amount) AS avg_total_amount,
    COUNT(1) AS total_records,
    COUNT(total_amount) AS not_null_total_number_of_records
FROM
    OPENROWSET(
        BULK 'trip_data_green_parquet/year=2020/month=01/',
        FORMAT = 'PARQUET',
        DATA_SOURCE = 'nyc_taxi_data_raw'    

    ) AS [result]

Check why totoal amount has negative values.

In [ ]:
SELECT
    payment_type, COUNT(1)
    
FROM
    OPENROWSET(
        BULK 'trip_data_green_parquet/year=2020/month=01/',
        FORMAT = 'PARQUET',
        DATA_SOURCE = 'nyc_taxi_data_raw'    

    ) AS [result]
    --WHERE total_amount < 0
    GROUP BY payment_type
    ORDER BY payment_type;

Check payment type

In [ ]:
SELECT payment_type, description
FROM
    OPENROWSET(
        BULK 'payment_type.json',
        DATA_SOURCE = 'nyc_taxi_data_raw',
        FORMAT = 'CSV',
        FIELDTERMINATOR = '0x0b',
        FIELDQUOTE = '0x0b'
    ) WITH(
        jsonDoc NVARCHAR(MAX)
    )AS payment_type
    CROSS APPLY OPENJSON(jsonDoc) --INNER JOIN WITHOUT ANY JOIN CONDITION.
    WITH(
        payment_type SMALLINT,
        description VARCHAR(20) '$.payment_type_desc'
    );